# 1 - Data analysis of CO2 under extreme conditions

In [1]:
# Loading all important libraries

import numpy as np
import scipy as scp
import dscribe as ds
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# folder stuff

home_folder=str("/home/moogmt");
co2_folder=home_folder+str("/CO2/CO2_AIMD/");

# Temperature and volumes

V=8.82;
T=3000;

# final folder

data_folder=co2_folder+str(V)+"/"+str(T)+"K/";

# final .xyz

filepath=data_folder+str("TRAJEC_wrapped.xyz");

In [8]:
# Useful functions

def getNbSteps( filepath ):
    # Values to return
    step_number=0
    nb_atoms=0
    # Reading file
    fp=open( filepath, "r" )
    read=fp.readline()
    if read != "" :
        nb_atoms=int(((read.rstrip("\n")).split())[0])
    while ( read != "" ) :
        step_number += 1
        read=fp.readline()
    fp.close()
    return nb_atoms, int(step_number/(nb_atoms+2))

def getTypes( filepath, nb_atoms ):
    # Initialize file
    types_names = ["" for x in range(nb_atoms)]
    fp=open( filepath, "r" )
    fp.readline()
    fp.readline()
    for atom in range(nb_atoms):
        types_names[atom]=((fp.readline().rstrip("\n")).split())[0]
    fp.close()
    types=[ types_names[1] ]
    types_vector=[0]
    for atom in range(nb_atoms):
        found = False 
        for i in range(len(types)):
            if types_names[atom] == types[i] :
                types_vector=np.append(types_vector,i)
                found = True
                break
        if not(found):
            types=np.append(types,types_names[atom])
            types_vector=np.append(types_vector,i)
    return types, types_vector

def readXYZ( filepath ):
    nb_atoms, nb_steps = getNbSteps( filepath )
    types, types_vector = getTypes( filepath, nb_atoms )
    positions=np.zeros(( nb_steps, nb_atoms, 3 ))        
    fp=open( filepath, "r" )
    for step in range( nb_steps ):
        # Reading two comments lines
        fp.readline()
        fp.readline()
        for atom in range( nb_atoms ):
            line=(fp.readline()).rstrip("\n").split()
            # Reading two comments lines
            for i in range(3):
                positions[step,atom,i] = float(line[i+1])
    fp.close()
    return positions, types, types_vector

def cellOrthoDistance( x1, x2, a):
    dx=x1-x2
    if dx > a*0.5:
        dx -= a
    if dx < -a*0.5:
        dx += a
    return dx*dx

def distance( position1, position2, cell_param ):
    dist=0
    for i in range(3):
        dist += cellOrthoDistance( position1[i], position2[i], cell_param[i] )
    return np.sqrt(dist)

def distanceMatrix( positions, nb_atoms, cell_lengths ):
    distance_matrix=np.zeros((nb_atoms,nb_atoms))
    for i in range(nb_atoms):
        for j in range(i+1,nb_atoms):
            distance_matrix[i,j]=distance( positions[i,:], positions[j,:], cell_lengths )
            distance_matrix[j,i]=distance_matrix[i,j]
    return distance_matrix

def writeData( filepath, data ):    
    fp=open( filepath, "w" )
    for i in range(np.shape(data)[0]):
        for j in range(np.shape(data)[1]):
            fp.write(str(data[i,j])+" ")
        fp.write("\n")
    fp.close()
    return 

def computeDensity( data, nb_points ):
    density=np.zeros(( nb_points ))
    mins=np.zeros((len(nb_points)))
    maxs=np.zeros((len(nb_points)))
    deltas=np.zeros((len(nb_points)))
    for i in range(len(nb_points)):
        mins[i] = np.min(data[:,i])
        maxs[i] = np.max(data[:,i])
        deltas[i] = (maxs[i]-mins[i])/nb_points[i]
    nb_data=len(data[:,0])
    for i in range(nb_data):
        indexs=np.zeros((3))
        for j in range(3):
            indexs[j]=(data[i,j]-mins[j])/deltas[j]-1
        density[int(indexs[0]),int(indexs[1]),int(indexs[2])] += 1
    return density/nb_data, mins, maxs, deltas


In [3]:
# Reading file
positions,types,types_vector=readXYZ(filepath)
nb_steps,nb_atoms,dim = np.shape(positions)

In [9]:
# Computing Data

max_neigh=4
nbC=32
nb_steps_max=500
nb_data=nb_steps_max*nbC
data=np.zeros((nb_data,max_neigh))

count=0
for step in range(nb_steps_max):
    distance_matrix=distanceMatrix(positions[step,:,:],nb_atoms,[V,V,V])
    for atom in range(nbC):
        indexs=np.argsort(distance_matrix[atom,:])
        distance_matrix[atom,:]=distance_matrix[atom,indexs]
        data[count,:]=distance_matrix[atom,1:max_neigh+1]
        count += 1


In [78]:
# Writting data to file
file_out=data_folder+str("data.dat");
writeData(file_out,data)

Et donc ici:

In [10]:
nb_boxes=[100,100,100,100]
density,mins,maxs,deltas=computeDensity(data,nb_boxes)

array([ 0.000125,  0.000125,  0.000125, ...,  0.000125,  0.000125,
        0.000125])